In [38]:
from data_patches import process_one_pcd
import torch
import numpy as np
from torch.utils.data import DataLoader
from data_patches import Patches_dataset
import torch.nn as nn
import torch.nn.functional as F

In [12]:
num_patches = 381
train_data = Patches_dataset(1,10)
# val_data = Patches_dataset(num_patches,1000)
# test_data = Patches_dataset(num_patches,1000)

train_loader = DataLoader(train_data, batch_size=2, shuffle=True) # shape: 2, 10, 24,3
# val_loader = DataLoader(val_data, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=16, shuffle=True)



in process_pcd: (100000, 3) (100000, 3) 0 29471
Full batch converged at iteration 31/100 with center shifts = tensor([5.4145e-06], dtype=torch.float64).
in process_pcd: (100000, 3) (100000, 3) 0 81933
Full batch converged at iteration 26/100 with center shifts = tensor([6.4353e-06], dtype=torch.float64).
in process_pcd: (100000, 3) (100000, 3) 0 13644
Full batch converged at iteration 17/100 with center shifts = tensor([8.7857e-06], dtype=torch.float64).
in process_pcd: (100000, 3) (100000, 3) 0 6929
Full batch converged at iteration 20/100 with center shifts = tensor([5.7544e-06], dtype=torch.float64).
in process_pcd: (100000, 3) (100000, 3) 0 34899
Full batch converged at iteration 27/100 with center shifts = tensor([9.0582e-06], dtype=torch.float64).
in process_pcd: (100000, 3) (100000, 3) 0 6740
Full batch converged at iteration 19/100 with center shifts = tensor([0.], dtype=torch.float64).
in process_pcd: (100000, 3) (100000, 3) 0 49494
Full batch converged at iteration 17/100 wit

In [56]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(128, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.conv4 = nn.Conv2d(64, 32, (2,3), 1)
        self.conv5 = nn.Conv2d(32, 16, (1,3), 1)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(16*3*14, 96)
        self.fc2 = nn.Linear(96, 48)
        self.fc3 = nn.Linear(48, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)

        return x

In [57]:
model = CNN()
inp = next(iter(train_loader)).float()
inp = inp.permute(0,3,1,2)
print(inp[:,:,:8,:].shape)
out = model(inp)

torch.Size([2, 3, 8, 24])


In [58]:
out.shape

torch.Size([2, 3])

In [59]:
out

tensor([[ 0.1171,  0.3016, -0.0604],
        [ 0.1170,  0.3013, -0.0607]], grad_fn=<AddmmBackward0>)

In [67]:
# loss func
def loss_func(pred, target):
    pred = torch.divide(pred, torch.linalg.norm(pred))
    target = torch.divide(target, torch.linalg.norm(target))
    dot = torch.abs(torch.diag(torch.dot(pred, target)))
    # loss = torch.mean(torch.acos(dot))   OR
    loss = 1 - torch.mean(dot)

In [66]:
torch.acos(torch.tensor(0.3))

tensor(1.2661)

In [44]:
#traning
epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in epochs:
    num_patches = 381
    train_data = Patches_dataset(1,10)
    # val_data = Patches_dataset(num_patches,1000)
    # test_data = Patches_dataset(num_patches,1000)

    train_loader = DataLoader(train_data, batch_size=2, shuffle=True) # shape: 2, 10, 24,3
    # val_loader = DataLoader(val_data, batch_size=16, shuffle=True)
    # test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

    for i, (data, norms) in enumerate(train_loader, 0):
        inputs = data.float()
        inputs = inputs.permute(0,3,1,2)
        norms = norms.float()
        norms = norms.permute(0,3,1,2)
        optimizer.zero_grad()
        out = model(inputs)
        loss = loss_func(out, inputs)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, loss: {loss.item()}')
        


1792